# Conhecendo a Requests

## Primeiros Passos

In [ ]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()
token = os.getenv('GITHUB_TOKEN')
user = os.getenv('USER_GITHUB')

In [ ]:
r = requests.get('https://api.github.com/events')

In [ ]:
r

## Explorando a biblioteca

In [ ]:
r.status_code

In [ ]:
r.url

In [ ]:
r.text

In [ ]:
r.json()

## Utilizando outros Endpoints

In [ ]:
r = requests.get('https://api.github.com/versions')
r.status_code

In [ ]:
r.json()

# Extraindo Dados

In [ ]:
# Requisição para um usuário específico do GitHub

r_user = requests.get(f'https://api.github.com/users/{user}')

print(r_user.status_code, "\n", r_user.json(), "\n", r_user.url)

In [ ]:
# Informações sobre o usuário no GitHub
print(f"""Nome: {r_user.json()['name']}
Nome de Usuário: {r_user.json()['login']}
Número de Repositórios Públicos: {r_user.json()['public_repos']}
Data de Criação: {r_user.json()['created_at']}""")

## Obtendo dados dos repositórios

In [ ]:
# especificando a versão da API
headers = {'X-GitHub-Api-Version': '2022-11-28'}

In [ ]:
api_base_url = 'https://api.github.com'
owner = 'amzn' #username de quem vamos extrair os dados
url = f'{api_base_url}/users/{owner}/repos'

In [ ]:
url

In [ ]:
response = requests.get(url, headers=headers)
response.status_code

In [ ]:
response.json()

In [ ]:
len(response.json())

## Autenticação

In [ ]:
access_token = token
headers = {'Authorization': f'Bearer {access_token}',
            'X-GitHub-Api-Version':   '2022-11-28'}

## Paginação
Percorrer cada uma das páginas para extrair os dados de cada uma delas

In [ ]:
api_base_url = 'https://api.github.com'
owner = 'amzn' #username de quem vamos extrair os dados
url = f'{api_base_url}/users/{owner}/repos'
url

In [ ]:
repos_list = []
for page_num in range(1, 6):  # Pega as primeiras 5 páginas
    try:
        url_page = f'{url}?page={page_num}'
        response = requests.get(url_page, headers=headers)
        repos_list.append(response.json())
    except Exception:
        repos_list.append(None)

In [ ]:
repos_list

In [ ]:
len(repos_list)

In [ ]:
len(repos_list[0])

# Desafio: Extraindo Dados

Durante essa etapa, devemos nos atentar a alguns detalhes:

1. criar uma lista vazia chamada followers_list para armazenar as informações sobre os seguidores;
2. utilizar um laço para iterar e extrair os dados dos seguidores existentes em cada uma das páginas (dica: caso você não tenha conhecimento de quantas páginas com seguidores existem, pesquise como utilizar o laço While True do Python);
3. acrescentar o parâmetro "page" na url, alterando o valor dele a cada iteração do laço que estiver sendo usado para paginação (lembre-se que a numeração das páginas sempre começam a partir do número 1);
4. enviar o token e a versão da API para o parâmetro headers no momento de fazer a requisição GET.

In [ ]:
import requests

username = 'amzn'
url = f"https://api.github.com/users/{username}/followers"

response = requests.get(url, headers=headers)
followers = response.json()

In [ ]:
access_token = token
headers = {'Authorization': f'Bearer {access_token}',
            'X-GitHub-Api-Version':   '2022-11-28'}

In [ ]:
followers_list = []
page_num = 1

url_page = f'{url}?page={page_num}'
r = requests.get(url, headers=headers)

while(r.status_code == 200 and r.json()):
    url_page = f'{url}?page={page_num}'
    try:
        followers_list.append(r.json())
        r = requests.get(url_page, headers=headers)
    except Exception:
        followers_list.append(None)
    page_num += 1

In [ ]:
len(followers_list)

In [ ]:
len(followers_list[0])

In [ ]:
len(followers_list[78])

In [ ]:
Total_seguidores = sum(len(page) for page in followers_list)
print(Total_seguidores)

# Transformando os Dados

## Nomes dos repositórios

In [ ]:
repos_list

In [ ]:
repos_list[0][2]['name']

In [ ]:
repos_name  =[]
for page in repos_list:
    for repo in page:
        repos_name.append(repo['name'])

In [ ]:
repos_name[:10]

In [ ]:
len(repos_name)

## Linguagens dos repositórios

In [ ]:
repos_list[1][1]['language']

In [ ]:
repos_language  =[]
for page in repos_list:
    for repo in page:
        repos_language.append(repo['language'])

In [ ]:
len(repos_language)

In [ ]:
dados_amz = pd.DataFrame({
    'Repository Name': repos_name,
    'Language': repos_language
})

In [ ]:
dados_amz

## Salvando o DataFrame

In [ ]:
dados_amz.to_csv('dados/amzn_repositories.csv', index=False)

## Desafio Transformando os dados

No desafio da aula 2 nós extraímos os dados de todos os seguidores da conta do Github da Amazon e armazenamos eles na lista "followers_list". Considerando isso, nossa tarefa agora é aplicar as transformações necessárias nos dados armazenados na lista followers_list para **selecionarmos apenas o nome de usuário de cada seguidor.**

Para concluirmos essa tarefa é importante:

1. entender como os dados dos seguidores estão estruturados na lista;
2. armazenar o nome de usuário de cada seguidor em uma lista;
3. conferir o tamanho da lista de nomes de usuário de seguidores para ter certeza que é igual ao número de seguidores apresentados no Github;
4. estruturar os dados em um DataFrame e salvá-los em CSV.

In [ ]:
followers_list[0][0]

In [ ]:
nomes_seguidores = []

for page in followers_list:
    for follower in page:
        nomes_seguidores.append(follower['login'])

In [ ]:
len(nomes_seguidores) == Total_seguidores

In [ ]:
seguidores = pd.DataFrame({
    'Follower Username': nomes_seguidores
})

seguidores.to_csv('dados/amzn_followers.csv', index=False)

# Armazenando os Dados 

In [ ]:
api_base_url = 'https://api.github.com'
url = f'{api_base_url}/user/repos'
url

In [ ]:
data = {
    'name' : 'linguagens-utilizadas',
    'description': 'Repositório para armazenar as linguagens de programação da Amazon.',
    'private': False
}

response = requests.post(url, headers=headers, json=data)
response.status_code

## Formato do Arquivo

In [ ]:
import base64

In [ ]:
with open('dados/amzn_repositories.csv', 'rb') as file:
    file_content = file.read()
    
encoded_content = base64.b64encode(file_content).decode('utf-8')

## Upload de arquivo com PUT

In [ ]:
api_base_url = 'https://api.github.com'
username = 'zettavyte'
repo = 'linguagens-utilizadas'
path = 'dados/amzn_repositories.csv'

url = f'{api_base_url}/repos/{username}/{repo}/contents/{path}'
url

In [ ]:
data = { 
    'message': 'Adicionando o arquivo amzn_repositories.csv',
    'content': encoded_content
}

response = requests.put(url, headers=headers, json=data)
response.status_code

## Desafio Armazenando os dados

Fazer um fork de um repositório é como criar uma cópia de um projeto que pertence a outra pessoa ou organização dentro do seu próprio perfil no GitHub. Isso permite que você trabalhe em uma versão do projeto sem afetar o original, podendo fazer modificações, correções de bugs e melhorias.
Após analisar a documentação, é importante não esquecermos de realizar alguns passos:

1. conferir o endpoint que deve ser utilizado para realização do fork;
2. acessar os repositórios públicos da Amazon, selecionar um deles para fazer o fork e salvar seu nome em uma variável;
3. enviar o token e a versão da API no momento de fazer a requisição.

In [ ]:
api_base_url = 'https://api.github.com'
owner = 'amzn'
forked_repo = 'xfer'

url = f'{api_base_url}/repos/{owner}/{forked_repo}/forks'
url

In [ ]:
post_fork = requests.post(url, headers=headers)
post_fork.status_code

### Evidência Fork de Xfer

![Fork](evidencias/evidencia_fork.png)

# Estruturando o código

## Criando uma classe - Transformar os códigos em duas classes
Verificar arquivo abaixo:

[Classes](dados_repos.py)

[Manipulação Repositórios](manipula_repos.py)




# Requisição DELETE


In [ ]:
owner = 'zettavyte'
repo = 'linguagens-utilizadas'
url = f'https://api.github.com/repos/{owner}/{repo}'

response = requests.delete(url, headers=headers)
print(f"{response.status_code}")